Using XArray to read tiled GeoTIFF datasets
===========================================

This notebook shows how to use XArray and Dask to process large GeoTIFF datasets efficiently.

Download Data
-------------

Lets download a sample GeoTIFF dataset

https://oin-hotosm.s3.amazonaws.com/5abae68e65bd8f00110f3e42/0/5abae68e65bd8f00110f3e43.tif

In [1]:
import os
if not os.path.exists('myfile.tif'):
    import requests
    response = requests.get('https://oin-hotosm.s3.amazonaws.com/5abae68e65bd8f00110f3e42/0/5abae68e65bd8f00110f3e43.tif')
    with open('myfile.tif', 'wb') as f:
        f.write(response.content)

## Look at metadata with XArray and Rasterio

In [8]:
my_file = '/home/guillaume/Téléchargements/tiles_30_T_XP_S2B_MSIL1C_20180814T105019_N0206_R051_T30TXP_20180814T145300.SAFE_GRANULE_L1C_T30TXP_A007513_20180814T105021_IMG_DATA_T30TXP_20180814T105019_B04.jp2'
my_file2 = '/home/guillaume/Téléchargements/tiles_30_T_XP_S2B_MSIL1C_20180814T105019_N0206_R051_T30TXP_20180814T145300.SAFE_GRANULE_L1C_T30TXP_A007513_20180814T105021_IMG_DATA_T30TXP_20180814T105019_B08.jp2'

import xarray as xr
xr.open_rasterio('/home/guillaume/Téléchargements/tiles_30_T_XP_S2B_MSIL1C_20180814T105019_N0206_R051_T30TXP_20180814T145300.SAFE_GRANULE_L1C_T30TXP_A007513_20180814T105021_IMG_DATA_T30TXP_20180814T105019_B04.jp2')  # this only reads metadata to start

<xarray.DataArray (band: 1, y: 10980, x: 10980)>
[120560400 values with dtype=uint16]
Coordinates:
  * band     (band) int64 1
  * y        (y) float64 4.9e+06 4.9e+06 4.9e+06 4.9e+06 4.9e+06 4.9e+06 ...
  * x        (x) float64 6e+05 6e+05 6e+05 6e+05 6e+05 6.001e+05 6.001e+05 ...
Attributes:
    transform:   (600000.0, 10.0, 0.0, 4900020.0, 0.0, -10.0)
    crs:         +init=epsg:32630
    res:         (10.0, 10.0)
    is_tiled:    1
    nodatavals:  (nan,)

In [3]:
import rasterio
img = rasterio.open(my_file)
img

<open RasterReader name='/home/guillaume/Téléchargements/tiles_30_T_XP_S2B_MSIL1C_20180814T105019_N0206_R051_T30TXP_20180814T145300.SAFE_GRANULE_L1C_T30TXP_A007513_20180814T105021_IMG_DATA_T30TXP_20180814T105019_B04.jp2' mode='r'>

In [4]:
img.is_tiled  # can we read this data in chunks?

True

In [5]:
set(img.block_shapes)  # what are the block shapes that we expect from this file?

{(1024, 1024)}

Great, this dataset is chunked by band (each band separate) and x/y blocks of size 512x512.  We'll want our Dask array chunks to be a bit bigger than this, but we'll use a clean mulitple.

Create lazy XArray dataset around GeoTIFF file
-------------------------------------

In [9]:
ds1 = xr.open_rasterio(my_file, 
                      chunks={'band': 1, 'x': 2048, 'y': 2048})
ds2 = xr.open_rasterio(my_file2, chunks={'band': 1, 'x': 2048, 'y': 2048})
ds = xr.concat([ds1,ds2], dim='band')
ds

<xarray.DataArray (band: 2, y: 10980, x: 10980)>
dask.array<shape=(2, 10980, 10980), dtype=uint16, chunksize=(1, 2048, 2048)>
Coordinates:
  * y        (y) float64 4.9e+06 4.9e+06 4.9e+06 4.9e+06 4.9e+06 4.9e+06 ...
  * x        (x) float64 6e+05 6e+05 6e+05 6e+05 6e+05 6.001e+05 6.001e+05 ...
  * band     (band) int64 1 1
Attributes:
    transform:   (600000.0, 10.0, 0.0, 4900020.0, 0.0, -10.0)
    crs:         +init=epsg:32630
    res:         (10.0, 10.0)
    is_tiled:    1
    nodatavals:  (nan,)

We note that the variables are dask arrays rather than numpy arrays

In [10]:
ds.variable.data

dask.array<concatenate, shape=(2, 10980, 10980), dtype=uint16, chunksize=(1, 2048, 2048)>

Optionally create a Dask Client
-------------------------------

I do this just to look at the dashboard during execution.  You don't need to run this though. Things will work just fine with the local thread pool scheduler.

In [11]:
from dask.distributed import Client
client = Client(processes=False)
client

Client Scheduler: inproc://192.168.0.19/7256/1 Dashboard: http://localhost:8787/status,Cluster Workers: 1 Cores: 8 Memory: 16.77 GB


In [13]:
ndvi = (ds[0,:,:] - ds[1,:,:]) / (ds[0,:,:] + ds[1,:,:])

In [14]:
ndvi = ndvi.persist()

/home/guillaume/outils/miniconda3/envs/dask_dev/lib/python3.6/site-packages/distributed/worker.py:840: RuntimeWarning: divide by zero encountered in true_divide
  result = function(*args, **kwargs)
/home/guillaume/outils/miniconda3/envs/dask_dev/lib/python3.6/site-packages/distributed/worker.py:840: RuntimeWarning: invalid value encountered in true_divide
  result = function(*args, **kwargs)


In [18]:
import matplotlib.pyplot as plt


AttributeError: module 'matplotlib' has no attribute 'imgshow'